<a href="https://colab.research.google.com/github/donnadamus/mlinapplabs/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import tqdm
import plotly.express as px

In [2]:
def swissRoll(samples=1000, noise=0.0):
    t = 1.5 * 3.14 * (1 + 2*tf.random.uniform((samples, 1)))
    x = t * tf.math.cos(t)
    y = t * tf.math.sin(t)
    X = tf.concat([x, y], axis=1)
    if noise > 0:
        X += noise * tf.random.normal((samples, 2))
    return X


In [3]:
def makeBlock(in_features, out_features):
    return models.Sequential([
        layers.Dense(out_features, input_shape=(in_features,), activation=None),
        layers.BatchNormalization(),
        layers.ReLU()
    ])

In [4]:
encoder = models.Sequential([
    makeBlock(2, 64),
    makeBlock(64, 64),
    makeBlock(64, 64),
    makeBlock(64, 64),
    makeBlock(64, 64),
    layers.Dense(4, activation=None)
])

decoder = models.Sequential([
    makeBlock(2, 64),
    makeBlock(64, 64),
    makeBlock(64, 64),
    makeBlock(64, 64),
    makeBlock(64, 64),
    layers.Dense(2, activation=None)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [5]:
@tf.function
def sp(latent):
    mean = latent[:, :2]
    logVar = latent[:, 2:]
    std = tf.exp(0.5 * logVar)
    latentSample = mean + std * tf.random.normal(tf.shape(std))
    kl = tf.reduce_mean(0.5 * tf.reduce_sum(tf.square(mean) + tf.exp(logVar) - logVar - 1, axis=-1))
    return latentSample, kl

@tf.function
def train_step(sample):
    with tf.GradientTape() as tape:
        latent = encoder(sample)
        latentSample, klLoss = sp(latent)
        y = decoder(latentSample)
        reconstructionLoss = tf.reduce_mean(tf.losses.mean_squared_error(sample, y))
        loss = reconstructionLoss + klLoss
    gradients = tape.gradient(loss, encoder.trainable_variables + decoder.trainable_variables)
    optimizer.apply_gradients(zip(gradients, encoder.trainable_variables + decoder.trainable_variables))
    return loss

In [9]:
loss = 0.0
# Training Loop
for i in (pbar := tqdm.tqdm(range(2000))):
    sample = swissRoll(1024, 0.5)
    loss = train_step(sample)
    pbar.set_postfix({'Loss': loss.numpy()})

100%|██████████| 2000/2000 [00:50<00:00, 39.54it/s, Loss=4.13]


In [12]:
# Generate data
sR = swissRoll(1000, 0.5)
latent = encoder(sR)
latent, _ = sp(latent)
latent = tf.random.normal(tf.shape(latent))
generatedData = decoder(latent).numpy()

# Plot
px.scatter(x=sR[:, 0], y=sR[:, 1], width=512, height=512, template='plotly_dark', range_x=[-15, 15], range_y=[-15, 15]).show()
px.scatter(x=generatedData[:, 0], y=generatedData[:, 1], width=512, height=512, template='plotly_dark', range_x=[-15, 15], range_y=[-15, 15]).show()